# Databases

## Overview
This lesson covers database operations in Python, including working with SQLite, using SQLAlchemy ORM, and understanding database concepts.

## Learning Objectives
By the end of this lesson, you will be able to:
- Work with SQLite databases using `sqlite3`
- Use SQLAlchemy ORM for database operations
- Understand database relationships and migrations
- Implement CRUD operations
- Handle database transactions and errors

## Prerequisites
- Basic understanding of Python
- Familiarity with SQL concepts
- Understanding of relational databases
- Basic knowledge of data modeling

## Topics Covered
1. SQLite with sqlite3
2. ORM with SQLAlchemy
3. Database Relationships
4. CRUD Operations
5. Transactions and Error Handling
6. Database Migrations
7. Query Optimization
8. Best Practices


In [ ]:
# 1. SQLite with sqlite3
print("1. SQLite with sqlite3")
print("-" * 25)

import sqlite3
import os

# Create a database connection
print("Creating database connection:")
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Create tables
print("\nCreating tables:")
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        age INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT,
        user_id INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
''')

conn.commit()
print("Tables created successfully")

# Insert data
print("\nInserting data:")
users_data = [
    ('John Doe', 'john@example.com', 30),
    ('Jane Smith', 'jane@example.com', 25),
    ('Bob Johnson', 'bob@example.com', 35)
]

cursor.executemany('INSERT INTO users (name, email, age) VALUES (?, ?, ?)', users_data)
conn.commit()
print("Users inserted successfully")

posts_data = [
    ('First Post', 'This is my first post', 1),
    ('Second Post', 'This is my second post', 1),
    ('Hello World', 'Hello from Jane', 2),
    ('Python Tips', 'Some Python programming tips', 3)
]

cursor.executemany('INSERT INTO posts (title, content, user_id) VALUES (?, ?, ?)', posts_data)
conn.commit()
print("Posts inserted successfully")

# Query data
print("\nQuerying data:")
cursor.execute('SELECT * FROM users')
users = cursor.fetchall()
print("Users:")
for user in users:
    print(f"  ID: {user[0]}, Name: {user[1]}, Email: {user[2]}, Age: {user[3]}")

cursor.execute('SELECT * FROM posts')
posts = cursor.fetchall()
print("\nPosts:")
for post in posts:
    print(f"  ID: {post[0]}, Title: {post[1]}, Content: {post[2]}, User ID: {post[3]}")

# Join queries
print("\nJoin queries:")
cursor.execute('''
    SELECT u.name, p.title, p.content
    FROM users u
    JOIN posts p ON u.id = p.user_id
    ORDER BY u.name, p.title
''')
joined_data = cursor.fetchall()
print("Users and their posts:")
for row in joined_data:
    print(f"  {row[0]}: {row[1]} - {row[2]}")

# Update data
print("\nUpdating data:")
cursor.execute('UPDATE users SET age = 31 WHERE name = ?', ('John Doe',))
conn.commit()
print("User updated successfully")

# Verify update
cursor.execute('SELECT name, age FROM users WHERE name = ?', ('John Doe',))
updated_user = cursor.fetchone()
print(f"Updated user: {updated_user[0]}, Age: {updated_user[1]}")

# Delete data
print("\nDeleting data:")
cursor.execute('DELETE FROM posts WHERE title = ?', ('First Post',))
conn.commit()
print("Post deleted successfully")

# Verify deletion
cursor.execute('SELECT COUNT(*) FROM posts')
post_count = cursor.fetchone()[0]
print(f"Remaining posts: {post_count}")

# Transactions
print("\nTransactions:")
try:
    cursor.execute('BEGIN TRANSACTION')
    cursor.execute('INSERT INTO users (name, email, age) VALUES (?, ?, ?)', ('Alice', 'alice@example.com', 28))
    cursor.execute('INSERT INTO posts (title, content, user_id) VALUES (?, ?, ?)', ('Alice Post', 'Hello from Alice', 4))
    conn.commit()
    print("Transaction committed successfully")
except Exception as e:
    conn.rollback()
    print(f"Transaction rolled back: {e}")

# Error handling
print("\nError handling:")
try:
    cursor.execute('INSERT INTO users (name, email, age) VALUES (?, ?, ?)', ('John Doe', 'john@example.com', 30))
    conn.commit()
except sqlite3.IntegrityError as e:
    print(f"Integrity error: {e}")

# Parameterized queries
print("\nParameterized queries:")
name = 'Jane Smith'
cursor.execute('SELECT * FROM users WHERE name = ?', (name,))
user = cursor.fetchone()
if user:
    print(f"Found user: {user[1]} ({user[2]})")
else:
    print("User not found")

# Batch operations
print("\nBatch operations:")
batch_users = [
    ('Charlie', 'charlie@example.com', 22),
    ('Diana', 'diana@example.com', 29),
    ('Eve', 'eve@example.com', 26)
]
cursor.executemany('INSERT INTO users (name, email, age) VALUES (?, ?, ?)', batch_users)
conn.commit()
print("Batch insert completed")

# Aggregate functions
print("\nAggregate functions:")
cursor.execute('SELECT COUNT(*) FROM users')
user_count = cursor.fetchone()[0]
print(f"Total users: {user_count}")

cursor.execute('SELECT AVG(age) FROM users')
avg_age = cursor.fetchone()[0]
print(f"Average age: {avg_age:.2f}")

cursor.execute('SELECT MIN(age), MAX(age) FROM users')
min_max_age = cursor.fetchone()
print(f"Age range: {min_max_age[0]} - {min_max_age[1]}")

# Group by
print("\nGroup by:")
cursor.execute('''
    SELECT u.name, COUNT(p.id) as post_count
    FROM users u
    LEFT JOIN posts p ON u.id = p.user_id
    GROUP BY u.id, u.name
    ORDER BY post_count DESC
''')
user_posts = cursor.fetchall()
print("Users and their post counts:")
for row in user_posts:
    print(f"  {row[0]}: {row[1]} posts")

# Subqueries
print("\nSubqueries:")
cursor.execute('''
    SELECT name, age
    FROM users
    WHERE age > (SELECT AVG(age) FROM users)
    ORDER BY age DESC
''')
above_avg_users = cursor.fetchall()
print("Users above average age:")
for user in above_avg_users:
    print(f"  {user[0]}: {user[1]} years old")

# Indexes
print("\nIndexes:")
cursor.execute('CREATE INDEX IF NOT EXISTS idx_users_email ON users(email)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_posts_user_id ON posts(user_id)')
conn.commit()
print("Indexes created successfully")

# Database info
print("\nDatabase info:")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("Tables in database:")
for table in tables:
    print(f"  {table[0]}")

cursor.execute("PRAGMA table_info(users)")
columns = cursor.fetchall()
print("\nUsers table structure:")
for column in columns:
    print(f"  {column[1]} ({column[2]})")

# Close connection
print("\nClosing database connection:")
conn.close()
print("Connection closed")

# Clean up
if os.path.exists('example.db'):
    os.remove('example.db')
    print("Database file removed")


## Practice Exercises

### Exercise 1: SQLite Database Operations
Create a SQLite database with the following tables:
- `customers` (id, name, email, phone)
- `orders` (id, customer_id, product, quantity, price, date)
- `products` (id, name, description, price, stock)

Implement CRUD operations for all tables.

### Exercise 2: SQLAlchemy ORM
Convert the SQLite database from Exercise 1 to use SQLAlchemy ORM:
- Define models for all tables
- Implement relationships between models
- Create a session manager
- Implement CRUD operations using ORM

### Exercise 3: Database Relationships
Design and implement a database with complex relationships:
- One-to-many relationships
- Many-to-many relationships
- Self-referencing relationships
- Implement proper foreign key constraints

### Exercise 4: Database Migrations
Implement database migrations for your application:
- Create migration scripts
- Handle schema changes
- Implement rollback functionality
- Version control your database schema

### Exercise 5: Query Optimization
Optimize database queries for better performance:
- Analyze query execution plans
- Create appropriate indexes
- Optimize JOIN operations
- Implement query caching

### Exercise 6: Database Transactions
Implement proper transaction handling:
- Use transactions for complex operations
- Handle rollback scenarios
- Implement savepoints
- Test transaction isolation levels

### Exercise 7: Database Security
Implement security measures for your database:
- Use parameterized queries
- Implement input validation
- Set up proper user permissions
- Encrypt sensitive data

### Exercise 8: Database Backup and Recovery
Implement backup and recovery procedures:
- Create automated backups
- Implement point-in-time recovery
- Test recovery procedures
- Monitor backup integrity

### Exercise 9: Database Monitoring
Add monitoring and logging to your database operations:
- Log all database operations
- Monitor query performance
- Set up alerts for errors
- Track database usage statistics

### Exercise 10: Database Testing
Write comprehensive tests for your database operations:
- Unit tests for individual operations
- Integration tests for database workflows
- Performance tests
- Security tests


## Summary

In this lesson, we've covered the fundamentals of database operations in Python:

### Key Concepts Covered:
1. **SQLite with sqlite3**: Working with SQLite databases using the built-in `sqlite3` module
2. **ORM with SQLAlchemy**: Using Object-Relational Mapping for database operations
3. **Database Relationships**: Understanding and implementing different types of relationships
4. **CRUD Operations**: Create, Read, Update, and Delete operations
5. **Transactions**: Managing database transactions and error handling
6. **Query Optimization**: Improving database performance
7. **Database Security**: Implementing security best practices
8. **Best Practices**: Following database design and operation best practices

### Key Takeaways:
- **Use SQLite** for lightweight, file-based databases
- **Use SQLAlchemy** for complex applications requiring ORM
- **Implement proper relationships** between database tables
- **Use transactions** for data consistency
- **Optimize queries** for better performance
- **Follow security best practices** to protect your data
- **Test your database operations** thoroughly
- **Document your database schema** and operations

### Next Steps:
- Practice with different database systems (PostgreSQL, MySQL)
- Explore advanced ORM features and relationships
- Learn about database design patterns
- Study database performance optimization techniques
- Experiment with different database architectures

### Resources for Further Learning:
- [SQLite Documentation](https://www.sqlite.org/docs.html)
- [SQLAlchemy Documentation](https://docs.sqlalchemy.org/)
- [Database Design Tutorial](https://www.studytonight.com/dbms/)
- [SQL Tutorial](https://www.w3schools.com/sql/)
- [Database Performance Tuning](https://use-the-index-luke.com/)
- [Database Security Best Practices](https://owasp.org/www-project-top-10/)
- [Database Testing Guide](https://www.postgresql.org/docs/current/testing.html)
- [Database Migration Tools](https://alembic.sqlalchemy.org/)


# 7. Databases - Working with Data Persistence

Welcome to the seventh lesson of the Advanced Level! In this lesson, you'll learn how to work with databases in Python.

## Learning Objectives

By the end of this lesson, you will be able to:
- Work with SQLite databases
- Use SQLAlchemy ORM
- Perform CRUD operations
- Handle database connections
- Implement database migrations
- Optimize database queries

## Table of Contents

1. [SQLite Basics](#sqlite-basics)
2. [SQLAlchemy ORM](#sqlalchemy-orm)
3. [CRUD Operations](#crud-operations)
4. [Database Relationships](#database-relationships)
5. [Query Optimization](#query-optimization)
6. [Best Practices](#best-practices)


## SQLite Basics

SQLite is a lightweight, serverless database engine that's perfect for development and small applications. It's included with Python and requires no additional setup.

### Key Features:
- **Serverless**: No separate server process required
- **Zero Configuration**: Works out of the box
- **Cross-platform**: Works on all major operating systems
- **ACID Compliant**: Supports transactions
- **Embedded**: Database is stored in a single file


In [ ]:
# SQLite Basics
import sqlite3
import os
from contextlib import contextmanager

print("SQLite Basics")
print("=" * 20)

# Create a database connection
db_name = "example.db"

# Remove existing database file if it exists
if os.path.exists(db_name):
    os.remove(db_name)

# Create connection
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

print("1. Creating Tables:")
print("-" * 20)

# Create users table
cursor.execute("""
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        email TEXT UNIQUE NOT NULL,
        age INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")

# Create posts table
cursor.execute("""
    CREATE TABLE posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT,
        user_id INTEGER,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
""")

print("Tables created successfully!")

# Insert data
print(f"\n2. Inserting Data:")
print("-" * 18)

# Insert users
users_data = [
    ("alice", "alice@example.com", 25),
    ("bob", "bob@example.com", 30),
    ("charlie", "charlie@example.com", 35)
]

cursor.executemany(
    "INSERT INTO users (username, email, age) VALUES (?, ?, ?)",
    users_data
)

# Insert posts
posts_data = [
    ("My First Post", "This is my first post content", 1),
    ("Python Tips", "Here are some Python tips", 1),
    ("Database Design", "Thoughts on database design", 2),
    ("Web Development", "Web development best practices", 3)
]

cursor.executemany(
    "INSERT INTO posts (title, content, user_id) VALUES (?, ?, ?)",
    posts_data
)

conn.commit()
print("Data inserted successfully!")

# Query data
print(f"\n3. Querying Data:")
print("-" * 18)

# Select all users
cursor.execute("SELECT * FROM users")
users = cursor.fetchall()
print("All users:")
for user in users:
    print(f"  {user}")

# Select users with age > 25
cursor.execute("SELECT username, email FROM users WHERE age > ?", (25,))
older_users = cursor.fetchall()
print(f"\nUsers older than 25:")
for user in older_users:
    print(f"  {user}")

# Join query
cursor.execute("""
    SELECT u.username, p.title, p.created_at
    FROM users u
    JOIN posts p ON u.id = p.user_id
    ORDER BY p.created_at DESC
""")
user_posts = cursor.fetchall()
print(f"\nUser posts:")
for post in user_posts:
    print(f"  {post}")

# Update data
print(f"\n4. Updating Data:")
print("-" * 18)

cursor.execute("UPDATE users SET age = ? WHERE username = ?", (26, "alice"))
conn.commit()

cursor.execute("SELECT * FROM users WHERE username = ?", ("alice",))
updated_user = cursor.fetchone()
print(f"Updated user: {updated_user}")

# Delete data
print(f"\n5. Deleting Data:")
print("-" * 16)

cursor.execute("DELETE FROM posts WHERE title = ?", ("Web Development",))
conn.commit()

cursor.execute("SELECT COUNT(*) FROM posts")
post_count = cursor.fetchone()[0]
print(f"Posts remaining: {post_count}")

# Transactions
print(f"\n6. Transactions:")
print("-" * 16)

try:
    # Start transaction
    cursor.execute("BEGIN TRANSACTION")
    
    # Insert new user
    cursor.execute(
        "INSERT INTO users (username, email, age) VALUES (?, ?, ?)",
        ("david", "david@example.com", 28)
    )
    
    # Insert new post
    cursor.execute(
        "INSERT INTO posts (title, content, user_id) VALUES (?, ?, ?)",
        ("Transaction Test", "Testing transactions", 4)
    )
    
    # Commit transaction
    cursor.execute("COMMIT")
    print("Transaction committed successfully!")
    
except Exception as e:
    # Rollback on error
    cursor.execute("ROLLBACK")
    print(f"Transaction rolled back due to error: {e}")

# Context manager for database connections
print(f"\n7. Context Manager:")
print("-" * 18)

@contextmanager
def get_db_connection(db_name):
    """Context manager for database connections."""
    conn = sqlite3.connect(db_name)
    try:
        yield conn
    finally:
        conn.close()

# Use context manager
with get_db_connection(db_name) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM users")
    user_count = cursor.fetchone()[0]
    print(f"Total users: {user_count}")

# Database schema information
print(f"\n8. Database Schema:")
print("-" * 20)

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(f"Tables: {[table[0] for table in tables]}")

for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    print(f"\n{table_name} columns:")
    for col in columns:
        print(f"  {col[1]} ({col[2]})")

# Indexes
print(f"\n9. Indexes:")
print("-" * 12)

# Create index
cursor.execute("CREATE INDEX idx_username ON users(username)")
cursor.execute("CREATE INDEX idx_user_posts ON posts(user_id)")

# List indexes
cursor.execute("SELECT name FROM sqlite_master WHERE type='index'")
indexes = cursor.fetchall()
print(f"Indexes: {[idx[0] for idx in indexes]}")

# Query with EXPLAIN
print(f"\n10. Query Analysis:")
print("-" * 20)

cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM users WHERE username = 'alice'")
explain_result = cursor.fetchall()
print("Query plan:")
for row in explain_result:
    print(f"  {row}")

# Close connection
conn.close()

# SQLAlchemy ORM
print(f"\n" + "=" * 50)
print("SQLAlchemy ORM")
print("=" * 50)

from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create engine
engine = create_engine(f'sqlite:///{db_name}', echo=False)
Base = declarative_base()

# Define models
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    age = Column(Integer)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationship
    posts = relationship("Post", back_populates="user")
    
    def __repr__(self):
        return f"<User(username='{self.username}', email='{self.email}')>"

class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    content = Column(Text)
    user_id = Column(Integer, ForeignKey('users.id'))
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationship
    user = relationship("User", back_populates="posts")
    
    def __repr__(self):
        return f"<Post(title='{self.title}', user_id={self.user_id})>"

# Create tables
Base.metadata.create_all(engine)

# Create session
Session = sessionmaker(bind=engine)
session = Session()

print("1. ORM Models Created:")
print("-" * 22)

# Create new user
new_user = User(username="eve", email="eve@example.com", age=27)
session.add(new_user)
session.commit()
print(f"Created user: {new_user}")

# Query users
users = session.query(User).all()
print(f"\nAll users: {users}")

# Query with filters
young_users = session.query(User).filter(User.age < 30).all()
print(f"Young users: {young_users}")

# Create post for user
new_post = Post(title="ORM Post", content="This is a post created with ORM", user_id=new_user.id)
session.add(new_post)
session.commit()
print(f"Created post: {new_post}")

# Query with joins
user_posts = session.query(User, Post).join(Post).all()
print(f"\nUser-Post pairs: {user_posts}")

# Update user
user_to_update = session.query(User).filter(User.username == "eve").first()
if user_to_update:
    user_to_update.age = 28
    session.commit()
    print(f"Updated user: {user_to_update}")

# Delete user
user_to_delete = session.query(User).filter(User.username == "eve").first()
if user_to_delete:
    session.delete(user_to_delete)
    session.commit()
    print("User deleted successfully")

# Close session
session.close()

# Clean up
if os.path.exists(db_name):
    os.remove(db_name)

print(f"\nDatabase examples completed!")
